In [2]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [3]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.memory import MemorySaver

In [4]:
tool = TavilySearchResults(max_results=2)
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

In [5]:
from uuid import uuid4
def reduce_messages(left: list[AnyMessage], right: list[AnyMessage]):
    for message in right:
        if not message.id:
            message.id = str(uuid4())
    merged = left.copy()
    for message in right:
        for i, existing in enumerate(merged):
            if existing.id == message.id:
                merged[i]=message
                break
        else:
            merged.append(message)
    return merged
    

In [6]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], reduce_messages]
    

In [7]:
class Agent:
    def __init__(self, model, tools, checkpointer, system= ""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.execute_action)
        graph.add_conditional_edges("llm",
                                    self.action_exists,
                                    {True:"action", False: END})
        graph.add_edge("action","llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer,
                                   interrupt_before=['action'])
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)
    def action_exists(self, messages: AgentState):
        tool_calls = messages['messages'][-1].tool_calls
        return(len(tool_calls)>0)
    def call_openai(self, messages: AgentState):
        messages = messages['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)]+messages
        response = self.model.invoke(messages)
        return {"messages": [response]}
    
    def execute_action(self, messages: AgentState):
        tool_calls = messages['messages'][-1].tool_calls
        responses = []
        for t in tool_calls:
            print ("Invoking tools {t}")
            response = self.tools[t['name']].invoke(t['args'])
            responses.append(ToolMessage(tool_call_id=t["id"], name= t['name'], content=str(response)))
        print ("Back to LLM") 
        return {"messages": responses} 
        

In [8]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

In [9]:
model = ChatOpenAI(model="gpt-3.5-turbo")
abot = Agent(model=model, tools=[tool], checkpointer=memory, system=prompt)

In [10]:
messages = [HumanMessage(content="Whats the weather in SF?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Mpa2YQnkCXs6KeNChHUbE6iG', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 152, 'total_tokens': 174, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-5f052fb9-9b1b-4450-a4ad-682ccdbd0a70-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_Mpa2YQnkCXs6KeNChHUbE6iG', 'type': 'tool_call'}], usage_metadata={'input_tokens': 152, 'output_tokens': 22, 'total_tokens': 174, 'input_token_details': {'audio': 0

In [11]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in SF?', additional_kwargs={}, response_metadata={}, id='f6ecb2e1-bc77-4960-9180-62ca11040ed2'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Mpa2YQnkCXs6KeNChHUbE6iG', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 152, 'total_tokens': 174, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-5f052fb9-9b1b-4450-a4ad-682ccdbd0a70-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_Mpa2Y

In [12]:
abot.graph.get_state(thread).next


('action',)

In [13]:
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print (v)

Invoking tools {t}
Back to LLM
{'messages': [ToolMessage(content='[{\'title\': \'Weather in San Francisco\', \'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.775, \'lon\': -122.4183, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1741989659, \'localtime\': \'2025-03-14 15:00\'}, \'current\': {\'last_updated_epoch\': 1741989600, \'last_updated\': \'2025-03-14 15:00\', \'temp_c\': 13.9, \'temp_f\': 57.0, \'is_day\': 1, \'condition\': {\'text\': \'Partly cloudy\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/day/116.png\', \'code\': 1003}, \'wind_mph\': 22.6, \'wind_kph\': 36.4, \'wind_degree\': 280, \'wind_dir\': \'W\', \'pressure_mb\': 1010.0, \'pressure_in\': 29.83, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 67, \'cloud\': 75, \'feelslike_c\': 11.4, \'feelslike_f\': 52.4, \'windchill_c\': 8.8, \'windchill_f\': 47.9, \'heatindex_c\

In [14]:
messages = [HumanMessage("How many IPL teams are there and who won it last time and who are the favourites this year. What was the playing eleven for the defending champions?")]

In [ ]:
thread = {"configurable":{"thread_id":"2"}}
for event in abot.graph.stream({"messages":messages}, thread):
    for v in event.values():
        print (v)
while abot.graph.get_state(thread).next:
    print (abot.graph.get_state(thread))
    _input = input("proceed?: ")
    if _input != 'y':
        print ("aborting...")
        break
    else:
        for event in abot.graph.stream(None, thread):
            for v in event.values():
                print (v)
        
        

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ZIbUWbZlnEeUPqtH5dyQdzDc', 'function': {'arguments': '{"query": "Number of IPL teams"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}, {'id': 'call_QeLQG2Q5nD19Mi3kVumivsfC', 'function': {'arguments': '{"query": "Current IPL champions"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}, {'id': 'call_lfY1OrCluBHCjQlCaWw7Hlcv', 'function': {'arguments': '{"query": "Favourites for IPL 2022"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}, {'id': 'call_dME6cFdKRy1YqKhCC9rqYeSG', 'function': {'arguments': '{"query": "Playing eleven for IPL defending champions"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 104, 'prompt_tokens': 176, 'total_tokens': 280, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_toke